In [21]:
%load_ext autoreload
%autoreload 2
import oogeso.io
import oogeso.plots as plots
import IPython.display
import ipywidgets
import pprint
import logging
logging.basicConfig(level="INFO",format="%(asctime)s %(levelname)s [%(name)s.%(funcName)s:%(lineno)d] %(message)s")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
energy_system = oogeso.io.read_data_from_yaml("../tests/test_data/testdata1.yaml")

In [23]:
simulator = oogeso.Simulator(data=energy_system)

In [24]:
yy=[""]+list(simulator.optimiser.component_objects(oogeso.optimiser.pyo.Constraint, active=True))
@ipywidgets.interact(constraint=yy)
def showdata(constraint):
    pprint.pprint(constraint.pprint(),width=1) if constraint!="" else print("")

interactive(children=(Dropdown(description='constraint', options=('', <pyomo.core.base.constraint.IndexedConst…

In [25]:
# Active objective(s):
for ob in simulator.optimiser.component_objects(oogeso.optimiser.pyo.Objective, active=True): 
    ob.pprint()

objObjective : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : (varDevicePenalty[source1,el,out,0] + varDevicePenalty[source1,el,out,1])/2


In [26]:
res = simulator.run_simulation(solver="cbc",time_range=[0,4])

100%|██████████| 4/4 [00:00<00:00,  6.97it/s]


In [27]:
try:
    IPython.display.Image(plots.plot_network(simulator,timestep=1).create_png())
except:
    pass

In [28]:
res_str = oogeso.dto.serialisation.serialize_oogeso_data(res)

In [29]:
res2 = oogeso.dto.serialisation.deserialize_oogeso_data(res_str)

In [30]:
(res.edge_flow["el1",]==15).all()

True

In [31]:
#oogeso.plots.plot_sum_power_mix(res,optimisation_model=simulator.optimiser,carrier="el")

In [32]:
res.co2_rate_per_dev.unstack("device")

device,dem,source1
time,,
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
